In [1]:
"""
1. 安装 YOLOX 环境 
"""
%cd /content/
!git clone https://github.com/Megvii-BaseDetection/YOLOX.git
%cd YOLOX
# 切换到 0.3.0 版本
!git checkout -b 0.3.0 0.3.0
%pip install -U pip && pip install -r requirements.txt
%pip install -v -e . 

/content
Cloning into 'YOLOX'...
remote: Enumerating objects: 1850, done.
remote: Counting objects: 100% (68/68), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 1850 (delta 17), reused 32 (delta 6), pack-reused 1782
Receiving objects: 100% (1850/1850), 6.91 MiB | 9.88 MiB/s, done.
Resolving deltas: 100% (1076/1076), done.
/content/YOLOX
Switched to a new branch '0.3.0'
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Using pip 23.0.1 from /usr/local/lib/python3.8/dist-packages/pip (python 3.8)
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/YOLOX
  Running command python setup.py egg_info
  running egg_info
  creating /tmp/pip-pip-egg-info-8zdezzu5/yolox.egg-info
  writing /tmp/pip-pip-egg-info-8zdezzu5/yolox.egg-info/PKG-INFO
  writing depen

In [5]:
"""
2. 下载 yolox_s 模型
"""
%cd /content/drive/MyDrive/yolox-map-demo/
%mkdir models
!pip install wget
# 下载
import  wget
wget.download('https://github.com/Megvii-BaseDetection/YOLOX/releases/download/0.1.1rc0/yolox_s.pth','models')

/content/drive/MyDrive/yolox-map-demo
mkdir: cannot create directory ‘models’: File exists
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9674 sha256=ca926419c6cf5dc076bd53999bc1351ef6350288586acb76b40ee3ccb5cf70a0
  Stored in directory: /root/.cache/pip/wheels/bd/a8/c3/3cf2c14a1837a4e04bd98631724e81f33f462d86a1d895fae0
Successfully built wget


'models/yolox_s.pth'

In [2]:
"""
3. 数据集生成
"""
%cd /content/drive/MyDrive/yolox-map-demo/datasets/
# 数据集 7:2:1
!python  labelimg2voc.py

/content/drive/MyDrive/yolox-map-demo/datasets
VOCdevkit/VOC2007/ImageSets/Main/train.txt
VOCdevkit/VOC2007/ImageSets/Main/val.txt
VOCdevkit/VOC2007/ImageSets/Main/trainval.txt
VOCdevkit/VOC2007/ImageSets/Main/test.txt


In [ ]:
"""
4. 训练参数文件
"""
%cd /content/drive/MyDrive/yolox-map-demo/
%cp ./YOLOX/exps/example/yolox_voc/yolox_voc_s.py  ./exps/
%cp ./YOLOX/tools/train.py ./

# 1. exps/yolox_s.py 变更数据集文件夹名称
# 2. train.py


In [5]:
"""
5. 训练数据
"""
%cd /content/drive/MyDrive/yolox-map-demo/
# 删除 YOLOX_outputs ,
#%rm -r YOLOX
#%rm -r YOLOX_outputs
# 训练
!python train.py -f exps/yolox_voc_s.py -d 1 -b 8 --fp16 -o -c models/yolox_s.pth

/content/drive/MyDrive/yolox-map-demo
2023-02-17 23:59:49 | INFO     | yolox.core.trainer:130 - args: Namespace(batch_size=8, cache=None, ckpt='models/yolox_s.pth', devices=1, dist_backend='nccl', dist_url=None, exp_file='exps/yolox_voc_s.py', experiment_name='yolox_voc_s', fp16=True, logger='tensorboard', machine_rank=0, name=None, num_machines=1, occupy=True, opts=[], resume=False, start_epoch=None)
2023-02-17 23:59:49 | INFO     | yolox.core.trainer:131 - exp value:
╒═══════════════════╤════════════════════════════╕
│ keys              │ values                     │
╞═══════════════════╪════════════════════════════╡
│ seed              │ None                       │
├───────────────────┼────────────────────────────┤
│ output_dir        │ './YOLOX_outputs'          │
├───────────────────┼────────────────────────────┤
│ print_interval    │ 10                         │
├───────────────────┼────────────────────────────┤
│ eval_interval     │ 10                         │
├───────────────

In [7]:
"""
6. 模型评估
"""
%cd /content/drive/MyDrive/yolox-map-demo/
%cp ./YOLOX_outputs/yolox_voc_s/best_ckpt.pth ./models/ 
%mv ./models/best_ckpt.pth ./models/voc_ckpt.pth
%cp ./YOLOX/tools/eval.py ./ 

# 评估命令
!python eval.py -f exps/yolox_voc_s.py -c models/voc_ckpt.pth -b 16 -d 1 --conf 0.001 --fp16 --fuse

/content/drive/MyDrive/yolox-map-demo
cp: cannot stat './YOLOX/tools/eval.py': No such file or directory
2023-02-18 00:09:18 | INFO     | __main__:139 - Args: Namespace(batch_size=16, ckpt='models/voc_ckpt.pth', conf=0.001, devices=1, dist_backend='nccl', dist_url=None, exp_file='exps/yolox_voc_s.py', experiment_name='yolox_voc_s', fp16=True, fuse=True, legacy=False, machine_rank=0, name=None, nms=None, num_machines=1, opts=[], seed=None, speed=False, test=False, trt=False, tsize=None)
2023-02-18 00:09:18 | INFO     | __main__:149 - Model Summary: Params: 8.94M, Gflops: 26.76
2023-02-18 00:09:18 | INFO     | __main__:150 - Model Structure:
YOLOX(
  (backbone): YOLOPAFPN(
    (backbone): CSPDarknet(
      (stem): Focus(
        (conv): BaseConv(
          (conv): Conv2d(12, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
      

In [10]:
"""
7. 图片测试 
"""
%cd /content/drive/MyDrive/yolox-map-demo/
# cp ./YOLOX/tools/demo.py ./ 
# 修改 demo.p/COCO_CLASS 内容
# 测试
!python demo.py image -f exps/yolox_voc_s.py -c models/voc_ckpt.pth --path assets/test.png --conf 0.25 --nms 0.45 --tsize 640 --save_result --device gpu

/content/drive/MyDrive/yolox-map-demo
2023-02-18 00:12:59.688 | INFO     | __main__:main:261 - Args: Namespace(camid=0, ckpt='models/voc_ckpt.pth', conf=0.25, demo='image', device='gpu', exp_file='exps/yolox_voc_s.py', experiment_name='yolox_voc_s', fp16=False, fuse=False, legacy=False, name=None, nms=0.45, path='assets/test.png', save_result=True, trt=False, tsize=640)
2023-02-18 00:13:00.036 | INFO     | __main__:main:271 - Model Summary: Params: 8.94M, Gflops: 26.76
2023-02-18 00:13:02.768 | INFO     | __main__:main:284 - loading checkpoint
2023-02-18 00:13:03.039 | INFO     | __main__:main:288 - loaded checkpoint done.
2023-02-18 00:13:05.379 | INFO     | __main__:inference:167 - Infer time: 1.4871s
2023-02-18 00:13:05.397 | INFO     | __main__:image_demo:204 - Saving detection result in ./YOLOX_outputs/yolox_voc_s/vis_res/2023_02_18_00_13_03/test.png


In [9]:
"""
8. 删除缓存
"""
%cd /content/drive/MyDrive/yolox-map-demo/
# 删除世代模型结果
%rm ./YOLOX_outputs/yolox_voc_s/epoch_*.pth


/content/drive/MyDrive/yolox-map-demo
